In [45]:
from dotenv import load_dotenv
import os
import requests
import pandas as pd
from datetime import datetime

In [5]:
load_dotenv()

secret_key = os.getenv("SECRET_KEY")

In [82]:
class MarketstackClient:
    def __init__(self, api_key, base_folder='daily_data'):
        self.api_key = api_key
        self.base_url = 'http://api.marketstack.com/v1/eod'
        self.base_folder = base_folder

    def fetch_stock_data(self, symbol, date_from, date_to,limit):
        params = {
            'access_key': self.api_key,
            'symbols': symbol,
            'limit': limit,
            'date_from': date_from,
            'date_to': date_to
        }
        response = requests.get(self.base_url, params=params)
        data = response.json()

        if 'data' not in data:
            raise Exception(f"Failed to fetch data: {data.get('error', 'Unknown error')}")

        return data['data']

    def save_data_by_date(self, data):
        # Create a DataFrame from the fetched data
        df = pd.DataFrame(data)
        
        # Convert the 'date' column to datetime and extract only the date
        df['date'] = pd.to_datetime(df['date']).dt.date
        
        # Group the DataFrame by the 'date' column and save each group to its own CSV file
        for date_val, group in df.groupby('date'):
            file_path = os.path.join(self.base_folder, f"{date_val}.csv")
            group.to_csv(file_path, index=False, header=True)
            print(f"Data for {date_val} saved to {file_path}")
        

    def get_latest_prices(self, symbol, limit=5):
        # Check if the folder exists, then read the latest data
        current_date = datetime.today().strftime('%Y-%m-%d')
        folder_path = os.path.join(self.base_folder, current_date)
        file_path = os.path.join(folder_path, f"{symbol}.csv")
        
        try:
            df = pd.read_csv(file_path)
            df['date'] = pd.to_datetime(df['date'])
            latest = df.sort_values(by='date', ascending=False).head(limit)
            return latest
        except FileNotFoundError:
            return f"No data found for {symbol} on {current_date}. Please fetch and save stock data first."


In [83]:
client = MarketstackClient(api_key=secret_key)

In [84]:
data = client.fetch_stock_data('TSLA,MSFT,AAPL,META', date_from = '2025-04-02', date_to = '2025-04-04', limit = 100)

In [80]:
data

[{'open': 193.89,
  'high': 199.88,
  'low': 187.34,
  'close': 188.38,
  'volume': 125569000.0,
  'adj_high': 199.88,
  'adj_low': 187.34,
  'adj_close': 188.38,
  'adj_open': 193.89,
  'adj_volume': 125910913.0,
  'split_factor': 1.0,
  'dividend': 0.0,
  'symbol': 'AAPL',
  'exchange': 'XNAS',
  'date': '2025-04-04T00:00:00+0000'},
 {'open': 506.62,
  'high': 518.0,
  'low': 494.2,
  'close': 504.73,
  'volume': 38589814.0,
  'adj_high': 518.0,
  'adj_low': 494.2,
  'adj_close': 504.73,
  'adj_open': 506.62,
  'adj_volume': 38589814.0,
  'split_factor': 1.0,
  'dividend': 0.0,
  'symbol': 'META',
  'exchange': 'ARCX',
  'date': '2025-04-04T00:00:00+0000'},
 {'open': 364.13,
  'high': 374.59,
  'low': 359.48,
  'close': 359.84,
  'volume': 49138700.0,
  'adj_high': 374.59,
  'adj_low': 359.48,
  'adj_close': 359.84,
  'adj_open': 364.125,
  'adj_volume': 49209854.0,
  'split_factor': 1.0,
  'dividend': 0.0,
  'symbol': 'MSFT',
  'exchange': 'XNAS',
  'date': '2025-04-04T00:00:00+0000

In [86]:
client.save_data_by_date(data)

Data for 2025-04-02 saved to daily_data\2025-04-02.csv
Data for 2025-04-03 saved to daily_data\2025-04-03.csv
Data for 2025-04-04 saved to daily_data\2025-04-04.csv
